In [3]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-m0ezpg8s
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-m0ezpg8s
  Resolved https://github.com/facebookresearch/detectron2.git to commit 0df2d73d0013db7de629602c23cc120219b4f2b8
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for detectron2: filename=detectron2-0.6-cp310-cp310-linux_x86_64.whl size=6172799 sha256=c8b7d770a18c63e0d2f88fcabf0

In [1]:
import detectron2
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

print("Detectron2 is successfully installed and imports are working.")

Detectron2 is successfully installed and imports are working.


In [2]:
!pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.9.0.80
    Uninstalling opencv-python-headless-4.9.0.80:
      Successfully uninstalled opencv-python-headless-4.9.0.80
  Attempting uninstall: idna
    Found existing installation: idna 3.7
    Uninstalling idna-3.7:
      Successfully uninstalled idna-3.7
  Attempting uninstall: cycler
    Found existing installation: cycler 0.12.1
    Uninstalling cycler-0.12.1:
      Successfully uninstalled cycler-0.

In [1]:
# !pip install ultralytics==8.0.196
# !pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="pSQhP0s4ioUPaV8pgbQC")
project = rf.workspace("food-detection-picix").project("food-detection-final-2")
version = project.version(1)
dataset = version.download("coco-segmentation")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Food-Detection-Final-2-1 in coco-segmentation:: 100%|██████████| 6254/6254 [00:01<00:00, 4713.33it/s]


In [2]:
# The dataset.location should contain the COCO formatted dataset now
train_json = f"{dataset.location}/train/_annotations.coco.json"
val_json = f"{dataset.location}/valid/_annotations.coco.json"
test_json = f"{dataset.location}/test/_annotations.coco.json"

In [3]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog

register_coco_instances("food_train", {}, train_json, f"{dataset.location}/train")
register_coco_instances("food_val", {}, val_json, f"{dataset.location}/valid")
register_coco_instances("food_test", {}, test_json, f"{dataset.location}/test")

food_metadata = MetadataCatalog.get("food_train")
dataset_dicts = DatasetCatalog.get("food_train")

Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



In [4]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.data import MetadataCatalog

# Initialize the config
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

# Dataset registration
cfg.DATASETS.TRAIN = ("food_train",)
cfg.DATASETS.TEST = ("food_val",)
cfg.DATALOADER.NUM_WORKERS = 2

# Model weights and configurations
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025

# Adjust based on your dataset size and desired number of epochs
dataset_size = 4423  # Number of images in your training dataset
batch_size = cfg.SOLVER.IMS_PER_BATCH
iterations_per_epoch = dataset_size // batch_size
number_of_epochs = 50
cfg.SOLVER.MAX_ITER = iterations_per_epoch * number_of_epochs

# Further model and training settings
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128  # Adjust based on your dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(food_metadata.thing_classes)  # Your number of classes

cfg.OUTPUT_DIR = "./output_mask_rcnn"


In [5]:
# Define augmentation using Detectron2's TransformGen
from detectron2.data import transforms as T

augmentation = [
    T.RandomRotation(angle=[-15, 15]),  # Rotation between -15 and +15 degrees
    T.Resize((640, 640)),               # Resize to 640x640
    T.RandomBrightness(intensity_min=0.8, intensity_max=1.2),  # Random brightness adjustment
    T.RandomContrast(intensity_min=0.8, intensity_max=1.2),    # Random contrast adjustment
    T.RandomSaturation(intensity_min=0.8, intensity_max=1.2),  # Random saturation adjustment
    T.RandomLighting(scale=0.2)                                # Random lighting adjustment
]

# Override the data loader to include the augmentations
from detectron2.data import DatasetMapper, build_detection_train_loader
from detectron2.engine import DefaultTrainer

class MyTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=DatasetMapper(cfg, is_train=True, augmentations=augmentation))



In [6]:
# Train the model
trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[06/02 21:31:41 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_f10217.pkl: 178MB [00:00, 319MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


[06/02 21:31:42 d2.engine.train_loop]: Starting training from iteration 0


torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)


Streaming output truncated to the last 5000 lines.
[06/02 21:54:24 d2.utils.events]:  eta: 3:21:16  iter: 11019  total_loss: 0.4948  loss_cls: 0.2336  loss_box_reg: 0.1294  loss_mask: 0.138  loss_rpn_cls: 0.0006852  loss_rpn_loc: 0.002724    time: 0.1219  last_time: 0.1189  data_time: 0.0041  last_data_time: 0.0037   lr: 0.00025  max_mem: 1280M
[06/02 21:54:27 d2.utils.events]:  eta: 3:21:07  iter: 11039  total_loss: 0.3813  loss_cls: 0.16  loss_box_reg: 0.08785  loss_mask: 0.1147  loss_rpn_cls: 0.000294  loss_rpn_loc: 0.002961    time: 0.1219  last_time: 0.1167  data_time: 0.0041  last_data_time: 0.0039   lr: 0.00025  max_mem: 1280M
[06/02 21:54:29 d2.utils.events]:  eta: 3:20:55  iter: 11059  total_loss: 0.4568  loss_cls: 0.186  loss_box_reg: 0.1264  loss_mask: 0.166  loss_rpn_cls: 0.0008757  loss_rpn_loc: 0.002847    time: 0.1219  last_time: 0.1257  data_time: 0.0039  last_data_time: 0.0039   lr: 0.00025  max_mem: 1280M
[06/02 21:54:31 d2.utils.events]:  eta: 3:20:47  iter: 11079  t

In [7]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

val_evaluator = COCOEvaluator("food_val", cfg, False, output_dir="./output_mask_rcnn")
val_loader = build_detection_test_loader(cfg, "food_val")
val_results = inference_on_dataset(trainer.model, val_loader, val_evaluator)
print("Validation Results:", val_results)

WARNING [06/03 01:17:51 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
WARNING [06/03 01:17:51 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[06/03 01:17:51 d2.data.datasets.coco]: Loaded 1216 images in COCO format from /content/Food-Detection-Final-2-1/valid/_annotations.coco.json
[06/03 01:17:51 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[06/03 01:17:51 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[06/03 01:17:51 d2.data.common]: Serializing 1216 elements to byte tensors and concatenating them all ...
[06/03 01:17:51 d2.data.common]: Serialized dataset takes 0.56 MiB
[06/03 01:17:51 d2.evaluation.evaluator]: Start inference on 1216 batches


os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.


[06/03 01:17:52 d2.evaluation.evaluator]: Inference done 11/1216. Dataloading: 0.0011 s/iter. Inference: 0.0402 s/iter. Eval: 0.0069 s/iter. Total: 0.0482 s/iter. ETA=0:00:58
[06/03 01:17:57 d2.evaluation.evaluator]: Inference done 129/1216. Dataloading: 0.0013 s/iter. Inference: 0.0373 s/iter. Eval: 0.0041 s/iter. Total: 0.0427 s/iter. ETA=0:00:46
[06/03 01:18:02 d2.evaluation.evaluator]: Inference done 248/1216. Dataloading: 0.0013 s/iter. Inference: 0.0369 s/iter. Eval: 0.0043 s/iter. Total: 0.0426 s/iter. ETA=0:00:41
[06/03 01:18:07 d2.evaluation.evaluator]: Inference done 365/1216. Dataloading: 0.0013 s/iter. Inference: 0.0369 s/iter. Eval: 0.0045 s/iter. Total: 0.0427 s/iter. ETA=0:00:36
[06/03 01:18:12 d2.evaluation.evaluator]: Inference done 483/1216. Dataloading: 0.0013 s/iter. Inference: 0.0369 s/iter. Eval: 0.0045 s/iter. Total: 0.0427 s/iter. ETA=0:00:31
[06/03 01:18:17 d2.evaluation.evaluator]: Inference done 602/1216. Dataloading: 0.0013 s/iter. Inference: 0.0368 s/iter. 

In [8]:
# Evaluate on the test dataset
cfg.DATASETS.TEST = ("food_test",)
test_evaluator = COCOEvaluator("food_test", cfg, False, output_dir="./output_mask_rcnn")
test_loader = build_detection_test_loader(cfg, "food_test")
test_results = inference_on_dataset(trainer.model, test_loader, test_evaluator)
print("Test Results:", test_results)

WARNING [06/03 01:18:45 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
WARNING [06/03 01:18:45 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[06/03 01:18:45 d2.data.datasets.coco]: Loaded 608 images in COCO format from /content/Food-Detection-Final-2-1/test/_annotations.coco.json
[06/03 01:18:45 d2.data.build]: Distribution of instances among all 18 categories:
|   category    | #instances   |   category    | #instances   |   category    | #instances   |
|:-------------:|:-------------|:-------------:|:-------------|:-------------:|:-------------|
| foods-Foods.. | 0            | Braised dish  | 31           | Chinese pan.. | 70           |
|  Fried food   | 65           | Northern cake | 46           |    Pickles    | 66           |
|  Steamed bun  | 262          | Stir-fried .. | 44           |      bao      | 23       

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.


[06/03 01:18:47 d2.evaluation.evaluator]: Inference done 44/608. Dataloading: 0.0014 s/iter. Inference: 0.0373 s/iter. Eval: 0.0053 s/iter. Total: 0.0440 s/iter. ETA=0:00:24
[06/03 01:18:52 d2.evaluation.evaluator]: Inference done 148/608. Dataloading: 0.0013 s/iter. Inference: 0.0405 s/iter. Eval: 0.0052 s/iter. Total: 0.0471 s/iter. ETA=0:00:21
[06/03 01:18:57 d2.evaluation.evaluator]: Inference done 265/608. Dataloading: 0.0013 s/iter. Inference: 0.0389 s/iter. Eval: 0.0048 s/iter. Total: 0.0452 s/iter. ETA=0:00:15
[06/03 01:19:02 d2.evaluation.evaluator]: Inference done 382/608. Dataloading: 0.0013 s/iter. Inference: 0.0384 s/iter. Eval: 0.0047 s/iter. Total: 0.0445 s/iter. ETA=0:00:10
[06/03 01:19:07 d2.evaluation.evaluator]: Inference done 498/608. Dataloading: 0.0013 s/iter. Inference: 0.0381 s/iter. Eval: 0.0048 s/iter. Total: 0.0442 s/iter. ETA=0:00:04
[06/03 01:19:12 d2.evaluation.evaluator]: Total inference time: 0:00:26.561494 (0.044049 s / iter per device, on 1 devices)
[0

In [9]:
# Output model parameters
model_parameters = trainer.model.state_dict()
for param_tensor in model_parameters:
    print(f"{param_tensor}: {model_parameters[param_tensor].size()}")

backbone.fpn_lateral2.weight: torch.Size([256, 256, 1, 1])
backbone.fpn_lateral2.bias: torch.Size([256])
backbone.fpn_output2.weight: torch.Size([256, 256, 3, 3])
backbone.fpn_output2.bias: torch.Size([256])
backbone.fpn_lateral3.weight: torch.Size([256, 512, 1, 1])
backbone.fpn_lateral3.bias: torch.Size([256])
backbone.fpn_output3.weight: torch.Size([256, 256, 3, 3])
backbone.fpn_output3.bias: torch.Size([256])
backbone.fpn_lateral4.weight: torch.Size([256, 1024, 1, 1])
backbone.fpn_lateral4.bias: torch.Size([256])
backbone.fpn_output4.weight: torch.Size([256, 256, 3, 3])
backbone.fpn_output4.bias: torch.Size([256])
backbone.fpn_lateral5.weight: torch.Size([256, 2048, 1, 1])
backbone.fpn_lateral5.bias: torch.Size([256])
backbone.fpn_output5.weight: torch.Size([256, 256, 3, 3])
backbone.fpn_output5.bias: torch.Size([256])
backbone.bottom_up.stem.conv1.weight: torch.Size([64, 3, 7, 7])
backbone.bottom_up.stem.conv1.norm.weight: torch.Size([64])
backbone.bottom_up.stem.conv1.norm.bias: t